
# Installing and importing dependencies




In [ ]:
!pip install attention
!pip install movecolumn
!pip install tensorflow_addons

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
import os,json
from google.colab import drive
import matplotlib.pyplot as plt
import string
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from sklearn.utils import shuffle
import statistics
import keras,tensorflow
from keras.layers import GRU, Flatten
from attention import Attention
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from keras.optimizers import Adam
from datetime import datetime, timedelta
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import tensorflow as tf
import movecolumn as mc
from keras.layers import Bidirectional 
from keras.layers import BatchNormalization
from keras.layers import Dropout
from tensorflow_addons.optimizers import AdamW
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import pickle
import os
from tensorflow.keras.regularizers import L1L2
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Input, Model 
from tensorflow.keras.layers import * 
from tensorflow.keras import backend 
from tensorflow.keras import utils
backend.clear_session()

drive.mount('/content/drive')

# Backend procesing


In [ ]:

def plot_autocorr(df):
    f, ax = plt.subplots(figsize=(10, 8))
    corr = df.corr()
    sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
                square=True, ax=ax,annot=True)
def read_dataframe(link):
  return pd.read_csv(link)
def minmaxavg(df):

  stats = df.groupby('dates')['NumArticles'].agg(['mean', 'max', 'min','count'])
  stats.columns = ['mean_NumArticles', 'max_NumArticles', 'min_NumArticles','NumberEvents']


  stats2 = df.groupby('dates')['GoldsteinScale'].agg(['mean', 'max', 'min',])
  stats2.columns = ['mean_GoldsteinScale', 'max_GoldsteinScale', 'min_GoldsteinScale',]

  stats = stats2.merge(stats, left_on = 'dates', right_index=True, how = 'left')
  
  stats3 = df.groupby('dates')['AvgTone'].agg(['mean', 'max', 'min'])
  stats3.columns = ['mean_AvgTone', 'max_AvgTone', 'min_AvgTone']
  
  stats = stats3.merge(stats, left_on = 'dates', right_index=True, how = 'left')
  

 
  stats4 = df.groupby('dates')['IsRootEvent'].agg(['mean', 'max', 'min','count'])
  stats4.columns = ['mean_IsRootEvent', 'max_IsRootEvent', 'min_IsRootEvent',"count_IsRootEvent"]
  stats = stats4.merge(stats, left_on = 'dates', right_index=True, how = 'left')
  
  stats4 = df.groupby('dates')['positive'].agg(['mean', 'max', 'min','count'])
  stats4.columns = ['mean_positive', 'max_positive', 'min_positive',"count_positive"]
  stats = stats4.merge(stats, left_on = 'dates', right_index=True, how = 'left')

  stats5 = df.groupby('dates')['negative'].agg(['mean', 'max', 'min','count'])
  stats5.columns = ['mean_negative', 'max_negative', 'min_negative',"count_negative"]
  stats = stats5.merge(stats, left_on = 'dates', right_index=True, how = 'left')

  stats6 = df.groupby('dates')['neutral'].agg(['mean', 'max', 'min','count'])
  stats6.columns = ['mean_neutral', 'max_neutral', 'min_neutral',"count_neutral"]
  stats = stats6.merge(stats, left_on = 'dates', right_index=True, how = 'left')

  lone_stats = stats
  gold = lone_stats['mean_NumArticles'].tolist()
  tone = lone_stats['mean_AvgTone'].tolist()
  
  res = []
  for i,j in zip(gold,tone):
    try:
      result = i / j
    except ZeroDivisionError:
      result = 0
   
    res.append(result)

  lone_stats = lone_stats.reset_index(level=0)
  lone_stats['articles_ratio_tone'] = res
 
  list_dates = lone_stats['dates'].tolist()
  last_week = []
  for day in list_dates:
    last_week.append(len(lone_stats[lone_stats['dates'] > day - pd.to_timedelta("7day")]))
  
  lone_stats['last_week'] = last_week
  last_week = []
  for day in list_dates:
    last_week.append(len(lone_stats[lone_stats['dates'] > day - pd.to_timedelta("14day")]))

  lone_stats['last_2weeks'] = last_week

  last_week = []
  for day in list_dates:
    last_week.append(len(lone_stats[lone_stats['dates'] > day - pd.to_timedelta("30day")]))

  lone_stats['last_month'] = last_week

 

  
  return stats,lone_stats

def count_sentiment(df):
  counting_df = pd.DataFrame()
  list_dates = df['dates'].tolist()
  topic_list = set(df['feelings'].tolist())
  
  listt = []
  point = 0
  for i in range(3):
    listt.append([])


 
  for day in list_dates:
    daily = df.loc[df['dates'] == day]
   
    for i in [0,1,2]:
   
      partial = daily.loc[daily['feelings'] == i]
    
      listt[i].append(len(partial))
    point += 1

  for i  in range(len(listt)):
    counting_df["feelings"+str(i)] = listt[i]

  counting_df["dates"] = list_dates
  return counting_df

def get_topics(df):

  cats = ['Social',"Política",'Migración / Racismo','Salud','Otros']
  cat = df['categories'].tolist()
  print(cat)
  per = []
  topics = []
  for dic in cat:

    res = list(eval(dic))
    per.append(res)

  list_topics = []
  for a in per:
    try:

      list_topics.append(a[0]['name'])
    except:
      list_topics.append("")
  return list_topics

def process_list(ls):
  final = []
  for l in ls:
    if 'Pol' in l:
      final.append("Politica")
    elif 'Salud' in l:
      final.append("Salud")
    elif 'Migración' in l:
       final.append("Migracion")
    elif 'Social' in l:
       final.append("Social")
    else:
      final.append("otros")
  return final


def group_by_topic(df):
  dfs = []
  topics = set(df['lista_topics'].tolist())
  for top in topics:
    dfs.append(df[df['lista_topics'] == top])
  return dfs

def group_titles_by_day(df):
  days = df['dates'].tolist()
  days = set(days)
  titles_matrix = []
  for day in days:

    partial = df[df['dates'] == day]
    titles_matrix.append(partial['titles'].tolist())
  return titles_matrix 


def group_titles_by_day(dates,df):
  days = set(dates)
  titles_matrix = []
  for day in days:
    partial = df[df['dates'] == day]
    titles_matrix.append(partial['titles'].tolist())
  return titles_matrix 
  
def calculate_rolling(df):
  keys = df.keys()
  days = [3,5,7,10,14,30,60,90]

  for key in keys:
      try:
        for day in days:
          key_df = key+"_"+str(day)
          df[key_df] = df[key].rolling(day).mean()
      except:
        continue
  return df

def count_fakes_in_day(df,df_maldita):
  dates = df['dates'].tolist()
  len_dates = []
  for date in dates:
  
    len_dates.append(len(df_maldita[df_maldita['just_date'] == str(date).split(" ")[0] ]))
  
  df['number_fake_news'] = len_dates

  return df



def create_dataset(df,days,flag):
  
  list_dates = df['dates'].tolist()
  quantity_fakes = df['number_fake_news'].tolist()
  reached_first = flag
  pointer = 0
  target_values = []
  dataset = []
  for date in list_dates:
    if reached_first == 1:
      d = date - timedelta(days=days)
      str_d = str(d).split(" ")[0]
      partial_df = df[(df['dates'] <= date) & (df['dates'] >= str_d)]
      get_val = df[(df['dates'] == date)]
      target_size = days + 1
      partial_df = partial_df.drop(columns=['dates','number_fake_news','just_dates'])
      if len(partial_df.values.tolist()) == target_size:
        dataset.append(partial_df.values.tolist())
        target_values.append(get_val['number_fake_news'].tolist()[0])
    else:
      if quantity_fakes[pointer] > 1000:
        reached_first = 1
      pointer += 1

  return dataset,target_values,list_dates





def prepare_data_consistency(dataset,target_values,scaler):
  
  train = dataset
  train_X = np.asarray(train).astype(np.float32)
  train_y = np.asarray(target_values).astype(np.float32)

  return train_X,train_y

def prepare_data(dataset,target_values,list_days,percentage):

  train = dataset
  train_X = np.asarray(train).astype(np.float32)
  train_y = np.asarray(target_values).astype(np.float32)
  diff_days = abs(len(train_X)-len(list_days))
  list_days = list_days[:len(list_days)-diff_days]
  X_test = train_X[int(len(train_X)*percentage):] 
  X_train= train_X[:int(len(train_X)*percentage)]
  y_test = train_y[int(len(train_y)*percentage):]
  y_train = train_y[:int(len(train_y)*percentage)]


  return X_test,X_train,y_test,y_train,scaler



def count_fakes_in_day(df,df_maldita):
  dates = df['dates'].tolist()
  len_dates = []
  for date in dates:
  
    len_dates.append(len(df_maldita[df_maldita['just_date'] == str(date).split(" ")[0] ]))
  
  
  df['number_fake_news'] = len_dates

  return df
  
def count_fakes_in_next_days(df,df_maldita,days):
  dates = df['dates'].tolist()
  len_dates = []
  for date in dates:    
    d = date + timedelta(days=days)

    str_d = str(d).split(" ")[0]
    original_date = str(date).split(" ")[0]

    testing_week = df_maldita[(df_maldita['just_date'] >= original_date) & (df_maldita['just_date'] < str_d)]

    len_dates.append(len(testing_week))
  
  df['number_fake_news'] = len_dates

  return df

def group_titles_by_day(dates,df):
  days = set(dates)
  
  titles_matrix = []
  for day in days:
    str_day = str(day).split(" ")[0]
    
    partial = df[df['dates'] == str_day]
    titles_matrix.append(partial['titles'].tolist())
  return titles_matrix 



def create_dataset_target_days(df,days):

  
  list_dates = df['dates'].tolist()
  quantity_fakes = df['number_fake_news'].tolist()

  reached_first = 1
  pointer = 0
  target_values = []
  dataset = []

  #if len(df) < 50:
  # reached_first = 1
  for date in list_dates:

    if reached_first == 1:
      

      d = date - timedelta(days=days)
      str_d = str(d).split(" ")[0]
      partial_df = df[(df['dates'] <= date) & (df['dates'] >= str_d)]
      get_val = df[(df['dates'] == date)]
      
      target_size = days + 1
      
      partial_df = partial_df.drop(columns=['dates','number_fake_news'])
      
      if len(partial_df.values.tolist()) == target_size:
        dataset.append(partial_df.values.tolist())
        target_values.append(get_val['number_fake_news'].tolist()[0])

    else:
      if quantity_fakes[pointer] > 1000:
        reached_first = 1
      pointer += 1


  return dataset,target_values,list_dates


# Models

In [ ]:
def create_basic_LSRM(train_X,train_y):
  
  model2 = Sequential()
  model2.add(LSTM(128,return_sequences=False,input_shape=(train_X.shape[1],train_X.shape[2])))
  model2.add(Dense(16))
  model2.add(Dense(8))
  model2.add(Dense(len(train_y[0]),activation='linear'))

  model2.compile(loss='mean_squared_error', optimizer='adam')
  return model2

In [ ]:
def autoencoder(train_X,train_y):

    enc_inputs = Input(shape=(train_X.shape[1], train_X.shape[2]), name='inputs')

    lstm = keras.layers.LSTM(units=64,return_sequences=True)(enc_inputs)
    lstm = keras.layers.LSTM(units=8,return_sequences=False)(lstm)
    gruencode = keras.layers.RepeatVector(train_X.shape[1], name='encoder_decoder_bridge')(lstm)
    lstm = keras.layers.LSTM(units=8,return_sequences=True)(gruencode)
    lstm = keras.layers.LSTM(units=64,return_sequences=False)(lstm)
    dense = keras.layers.Dense(units=32, activation='linear')(lstm)
    dense = Dropout(0.1)(dense)
    dense = keras.layers.Dense(units=16, activation='linear')(dense)
    dense = Dropout(0.1)(dense)
    dense = keras.layers.Dense(units=8, activation='linear')(dense)
    dense = Dropout(0.1)(dense)
    dense = keras.layers.Dense(units=len(train_y[0]), activation='linear')(dense)
    
    model = Model(enc_inputs, dense, name='encoder')
    model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mse'])

    return model

In [ ]:
def create_basic_model_enc_attention(train_X,train_y):
    n_hidden = 128
    enc_inputs = Input(shape=(train_X.shape[1], train_X.shape[2]), name='inputs')
    encoder_last_h1 = LSTM(
    n_hidden, dropout=0.2, recurrent_dropout=0.2, 
    return_sequences=True, return_state=False)(enc_inputs)
    x = Attention(units=64)(encoder_last_h1)
    mlp = Dense(64)(x)
    mlp = Dense(32)(mlp)
    mlp = Dense(16)(mlp)
    mlp = Dense(8)(mlp)
    mlp = Dense(len(train_y[0]))(mlp)
    model = Model(enc_inputs, mlp, name='encoder')
    opt = Adam(lr=0.01, clipnorm=1)
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

# Headlines model

In [ ]:
def create_model_text(X_train,vectorize_layersfinal,train_y):

    enc_inputs = Input(shape=(X_train.shape[1], X_train.shape[2]), name='inputs')
      
    num_features = 10
    embedding_dim = 128

    texts_input1 = Input(shape=(1,), dtype=tf.string)
    texts_input2 = Input(shape=(1,), dtype=tf.string)
    texts_input3 = Input(shape=(1,), dtype=tf.string)
    texts_input4 = Input(shape=(1,), dtype=tf.string)
    texts_input5 = Input(shape=(1,), dtype=tf.string)
    texts_input6 = Input(shape=(1,), dtype=tf.string)
    texts_input7 = Input(shape=(1,), dtype=tf.string)
    texts_input8 = Input(shape=(1,), dtype=tf.string)
    texts_input9 = Input(shape=(1,), dtype=tf.string)
    texts_input10 = Input(shape=(1,), dtype=tf.string)

    vect1 = vectorize_layersfinal(texts_input1)
    vect2 = vectorize_layersfinal(texts_input2)
    vect3 = vectorize_layersfinal(texts_input3)
    vect4 = vectorize_layersfinal(texts_input4)
    vect5 = vectorize_layersfinal(texts_input5)
    vect6 = vectorize_layersfinal(texts_input6)
    vect7 = vectorize_layersfinal(texts_input7)
    vect8 = vectorize_layersfinal(texts_input8)
    vect9 = vectorize_layersfinal(texts_input9)
    vect10 = vectorize_layersfinal(texts_input10)


    test_emb = Embedding(10000, embedding_dim, name = 'embedding1')
    emb1 = test_emb(vect1)  
    emb2 = test_emb(vect2)  
    emb3 = test_emb(vect3)  
    emb4 = test_emb(vect4)  
    emb5 = test_emb(vect5)  
    emb6 = test_emb(vect6)  
    emb7 = test_emb(vect7)  
    emb8 = test_emb(vect8)  
    emb9 = test_emb(vect9)  
    emb10 = test_emb(vect10)  

    msg_out1 = LSTM(32, kernel_regularizer = L1L2(l1=0.001, l2=0.01),dropout=0.3,return_sequences=False)(emb1)    
    msg_out2 = LSTM(32, kernel_regularizer = L1L2(l1=0.001, l2=0.01),dropout=0.3,return_sequences=False)(emb2)    
    msg_out3 = LSTM(32, kernel_regularizer = L1L2(l1=0.001, l2=0.01),dropout=0.3,return_sequences=False)(emb3)    
    msg_out4 = LSTM(32, kernel_regularizer = L1L2(l1=0.001, l2=0.01),dropout=0.3,return_sequences=False)(emb4)    
    msg_out5 = LSTM(32, kernel_regularizer = L1L2(l1=0.001, l2=0.01),dropout=0.3,return_sequences=False)(emb5)   
    msg_out6 = LSTM(32, kernel_regularizer = L1L2(l1=0.001, l2=0.01),dropout=0.3,return_sequences=False)(emb6)    
    msg_out7 = LSTM(32, kernel_regularizer = L1L2(l1=0.001, l2=0.01),dropout=0.3,return_sequences=False)(emb7)    
    msg_out8 = LSTM(32, kernel_regularizer = L1L2(l1=0.001, l2=0.01),dropout=0.3,return_sequences=False)(emb8)    
    msg_out9 = LSTM(32, kernel_regularizer = L1L2(l1=0.001, l2=0.01),dropout=0.3,return_sequences=False)(emb9)    
    msg_out10 = LSTM(32, kernel_regularizer = L1L2(l1=0.001, l2=0.01),dropout=0.3,return_sequences=False)(emb10)     

    gruencode = LSTM(256 ,return_sequences=False)(enc_inputs)
    x = concatenate([msg_out1,msg_out2,msg_out3,msg_out4,msg_out5,msg_out6,msg_out7,msg_out8,msg_out9,msg_out10, gruencode]) 
    mlp = Dense(1024, activation='linear')(x)
    mlp = Dropout(0.5)(mlp)
    mlp = Dense(512, activation='linear')(mlp)
    mlp = Dropout(0.3)(mlp)
    mlp = Dense(256, activation='linear')(mlp)
    mlp = Dropout(0.2)(mlp)
    mlp = Dense(128, activation='linear')(mlp)
    mlp = Dropout(0.2)(mlp)
    mlp = Dense(64, activation='linear')(mlp)
    mlp = Dropout(0.1)(mlp)
    mlp = Dense(32, activation='linear')(mlp)
    mlp = Dropout(0.1)(mlp)
    mlp = Dense(16, activation='linear')(mlp)
    mlp = Dropout(0.1)(mlp)
    mlp = Dense(len(train_y[0]), activation='linear',kernel_constraint=tf.keras.constraints.non_neg())(mlp)

    model = Model([enc_inputs,texts_input1,texts_input2,texts_input3,texts_input4,texts_input5,texts_input6,texts_input7,texts_input8,texts_input9,texts_input10], mlp, name='encoder')
    
    model.compile(loss='mse', optimizer='adam')
    return model


In [ ]:
training_gdelt = read_dataframe("/xxxx.csv")
training_maldita = read_dataframe("xxxxx.csv")
gtopL = get_topics(training_maldita)
gtopL = process_list(gtopL)
training_maldita['lista_topics'] = gtopL
training_gdelt = training_gdelt.drop_duplicates(subset=["titles"], keep=False)
training_gdelt = training_gdelt.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'GLOBALEVENTID',
      'FractionDate', 'Actor1Code',
       'Actor1CountryCode', 'Actor1KnownGroupCode', 'Actor1EthnicCode',
       'Actor1Religion1Code', 'Actor1Religion2Code', 'Actor1Type1Code',
       'Actor1Type2Code', 'Actor1Type3Code', 'Actor2Type1Code',
       'Actor2Type2Code', 'Actor2Type3Code', 
       'Actor1Geo_FeatureID', 'Actor2Geo_Type', 'Actor2Geo_FullName',
       'Actor2Geo_CountryCode', 'Actor2Geo_ADM1Code', 'Actor2Geo_Lat',
       'Actor2Geo_Long', 'Actor2Geo_FeatureID', 'ActionGeo_Type',
       'ActionGeo_FullName', 'ActionGeo_CountryCode', 'ActionGeo_ADM1Code',
       'ActionGeo_Lat', 'ActionGeo_Long', 'ActionGeo_FeatureID', 
       'SOURCEURL', 'Actor1Geo_ADM2Code', 'Actor2_ADM1Code', 'Actor3Geo_Type',
       'Actor3Geo_FullName', 'Randomplace'])

dates = training_gdelt['DATEADDED'].tolist()
new_dates = []
for date in dates:
  new_dates.append(str(date)[0:4]+"-"+str(date)[4:6]+"-"+str(date)[6:8])
training_gdelt['dates'] = new_dates
training_gdelt['dates'] = pd.to_datetime(training_gdelt['dates'],  format='%Y-%m-%d')
training_gdelt = training_gdelt.sort_values(by="dates")
get_titles = training_gdelt[training_gdelt['dates']>= "2019-09-23"]
training_gdelt = training_gdelt.sort_values(by="dates")
get_titles = get_titles[get_titles['dates']<= "2022-07-14"]
listdays = try_df['dates'].tolist()
texts_in_dataset = group_titles_by_day(listdays,training_gdelt)
texts_by_day = []
for textday in texts_in_dataset:
  sub_arra = []
  i = 0
  while i < 10:
    if len(textday) < 10:
      for x in range(len(textday)):
        sub_arra.append(textday[x].replace("_"," "))
        i += 1
      
      howmany = abs(len(sub_arra)-10)
      for y in range(howmany):
        sub_arra.append("")
    else:
      sub_arra.append(textday[i].replace("_"," "))
      i += 1
  
  texts_by_day.append(sub_arra)

In [ ]:
def create_dataset_text(df,days,flag,texts):
  
  list_dates = df['dates'].tolist()
  quantity_fakes = df['number_fake_news'].tolist()

  reached_first = flag
  pointer = 0
  target_values = []
  dataset = []
  sizee = 0
  new_texts = []
  

  for date in list_dates:
    if reached_first == 1:
      textos = []
      d = date - timedelta(days=days)
      str_d = str(d).split(" ")[0]

      partial_df = df[(df['dates'] <= date) & (df['dates'] >= str_d)]
      get_val = df[(df['dates'] == date)]
      
      target_size = days + 1
      
      partial_df = partial_df.drop(columns=['dates','number_fake_news'])

      if len(partial_df.values.tolist()) == target_size:
        dataset.append(partial_df.values.tolist())

        for i in range(days+1):
          try:
            textos.append(texts[pointer])
            pointer = pointer + 1
       
          except:
            textos.append("")
            pointer = pointer + 1
            
          
        
      
        sizee += 1

        target_values.append(get_val['number_fake_news'].tolist()[0])

    else:
      if quantity_fakes[pointer] > 1000:
        reached_first = 1
      pointer += 1

  return dataset,target_values,list_dates,texts

def prepare_data_with_texts(dataset,target_values,list_days,texts,percent):

  train = dataset
  train_X = np.asarray(train).astype(np.float32)
  train_y = np.asarray(target_values).astype(np.float32)

  train_X, train_y = shuffle(train_X, train_y, random_state=0)
  aaa, texts = shuffle(train_X, texts, random_state=0)
  diff_days = abs(len(train_X)-len(list_days))

  list_days = list_days[:len(list_days)-diff_days]

  X_test = train_X[int(len(train_X)*percent):] 
  X_train= train_X[:int(len(train_X)*percent)]

  y_test = train_y[int(len(train_y)*percent):]
  y_train = train_y[:int(len(train_y)*percent)]

  test_texts =  texts[int(len(texts)*percent):]
  train_texts = texts[:int(len(texts)*percent)]


  return X_test,X_train,y_test,y_train,scaler,test_texts,train_texts

In [ ]:
make_linear = []
for group in texts_by_day:
  for phrase in group:
    make_linear.append(phrase)

vectorize_layer_for_model = TextVectorization(
max_tokens=10000,
  output_mode='int',
  output_sequence_length=64)

vectorize_layer_for_model.adapt(make_linear)

In [ ]:
for percent in [0.6,0.9]  
  for j in [1,2,4,7]:
    days_to_predict = j
    epochs = 3000
    batch_size = 512
    texts_to_input = 1
    percentage_training = percent
    consistency = 0
    test_name = "xxx" +str(days_to_predict)+"_"+str(epochs)+"_"+str(percentage_training)
    try_df = count_fakes_in_next_days(to_train1,result,days_to_predict)
    try_df = mc.MoveToLast(try_df,'number_fake_news')
    try_df = try_df[try_df['dates']>= "2019-10-01"]
    try_df = try_df[try_df['dates']<= "2022-07-14"]
    dates_df = try_df['dates']
    try_df = try_df.drop(columns=["just_dates","dates"])


    train = try_df
    scalers={}
    for i in try_df.columns:
        
          scaler = MinMaxScaler(feature_range=(0,1))
          s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
          s_s=np.reshape(s_s,len(s_s))
          scalers['scaler_'+ i] = scaler
          train[i]=s_s

    train['dates'] = dates_df
    if consistency:
        
      testing_consistency = train[train['dates']>= "2022-02-01"]
      testing_consistency = testing_consistency[testing_consistency['dates']<= "2022-02-28"]
      dates_df = testing_consistency['dates']
      testing_consistency = testing_consistency.drop(columns=["dates"])
        
      for i in testing_consistency.columns:
          
            scaler = scalers['scaler_'+ i]
            s_s = scaler.fit_transform(testing_consistency[i].values.reshape(-1,1))
            s_s=np.reshape(s_s,len(s_s))
            testing_consistency[i]=s_s
      testing_consistency['dates'] = dates_df
      train = train[~train.dates.isin(testing_consistency.dates)]
      testing_consistency = mc.MoveToLast(testing_consistency,'number_fake_news')
    for i in [1,2,3,4,5,6,7,8,9,10]:
      err = []
      errrmse = []
      errmse = []
      errmae = []
      errr2= []
      days = i
      errrpear = []
      errors = []
        
      for i in range(5):
        lowest = 9999
        results_experiments = []
        
        try_df = mc.MoveToLast(try_df,'number_fake_news')
        listdays = try_df['dates'].tolist()
        texts_in_dataset = group_titles_by_day(listdays,training_gdelt)
        texts_by_day = []
        
        path = "xxxx"+test_name+"/"+test_name+str(days)+"/"
        if not os.path.exists("xxx"+test_name+"/"):
          os.mkdir("xxx"+test_name+"/")
        if not os.path.exists("xxx"+test_name+"/"+test_name+str(days)+"/"):
          os.mkdir("xxx"+test_name+"/"+test_name+str(days)+"/")

        for textday in texts_in_dataset:
          sub_arra = []
          i = 0
          while i < 10:
            if len(textday) < 10:
              for x in range(len(textday)):
                sub_arra += textday[x]
                i += 1
              
              howmany = abs(len(sub_arra)-10)
              for y in range(howmany):
                sub_arra.append("")
            else:
              sub_arra.append(textday[i])
              i += 1
          
          texts_by_day.append(sub_arra)
        target_days = i
        dataset,target_values,list_days,aa = create_dataset_text(try_df,target_days,1,texts_by_day)

        aa = aa[:len(aa)-target_days]

        dataset = np.array(dataset)
        target_values = np.array(target_values)
        X_test,X_train,y_test,y_train,scaler,test_texts,train_texts = prepare_data_with_texts(dataset,target_values,list_days,aa,percentage_training)
        train_text_new = np.array(train_texts)
        test_text_new = np.array(test_texts)


        train_1 = []
        train_2 = []
        train_3 = []
        train_4 = []
        train_5 = []
        train_6 = []
        train_7 = []
        train_8 = []
        train_9 = []
        train_10 = []

        test_1 = []
        test_2 = []
        test_3 = []
        test_4 = []
        test_5 = []
        test_6 = []
        test_7 = []
        test_8 = []
        test_9 = []
        test_10 = []

        for group in train_text_new:
          train_1.append(group[0])
          train_2.append(group[1])
          train_3.append(group[2])
          train_4.append(group[3])
          train_5.append(group[4])
          train_6.append(group[5])
          train_7.append(group[6])
          train_8.append(group[7])
          train_9.append(group[8])
          train_10.append(group[9])

        for group in test_text_new:
          test_1.append(group[0])
          test_2.append(group[1])
          test_3.append(group[2])
          test_4.append(group[3])
          test_5.append(group[4])
          test_6.append(group[5])
          test_7.append(group[6])
          test_8.append(group[7])
          test_9.append(group[8])
          test_10.append(group[9])

        train_1 = np.array(train_1)
        train_2 = np.array(train_2)
        train_3 = np.array(train_3)
        train_4 = np.array(train_4)
        train_5 = np.array(train_5)
        train_6 = np.array(train_6)
        train_7 = np.array(train_7)
        train_8 = np.array(train_8)
        train_9 = np.array(train_9)
        train_10 = np.array(train_10)


        test_1 = np.array(test_1)
        test_2 = np.array(test_2)
        test_3 = np.array(test_3)
        test_4 = np.array(test_4)
        test_5 = np.array(test_5)
        test_6 = np.array(test_6)
        test_7 = np.array(test_7)
        test_8 = np.array(test_8)
        test_9 = np.array(test_9)
        test_10 = np.array(test_10)

        early_stopping_monitor = EarlyStopping(
            monitor='val_loss',
            min_delta=0,
            patience=1000,
            verbose=0,
            mode='auto',
            baseline=None,
            restore_best_weights=True
        )




        model1 = create_model_text(X_train,vectorize_layer_for_model)
        history = model1.fit([X_train,train_1,train_2,train_3,train_4,train_5,train_6,train_7,train_8,train_9,train_10], y_train, epochs=3000, batch_size=batch_size,callbacks=[early_stopping_monitor],validation_data=([X_test,test_1,test_2,test_3,test_4,test_5,test_6,test_7,test_8,test_9,test_10], y_test), verbose=0, shuffle=True)
        yhat = model1.predict([X_test,test_1,test_2,test_3,test_4,test_5,test_6,test_7,test_8,test_9,test_10])
        yhat = yhat.flatten()
        y_testw = y_test.flatten()
        y_testw = scalers['scaler_number_fake_news'].inverse_transform(y_test.reshape(-1, 1)).flatten()
        yhat = scalers['scaler_number_fake_news'].inverse_transform(yhat.reshape(-1, 1)).flatten()
        yhat[yhat < 0] = 0
        rmse = sqrt(mean_squared_error(y_testw,yhat))
        mse = mean_squared_error(y_testw,yhat)
        mae = np.mean(np.abs(y_testw - yhat))
        r2 = r2_score(y_testw, yhat)
        cor = median_abs_deviation(y_testw - yhat) 
        with open("xxxx"+test_name+"/"+test_name+str(days)+'/'+str(i)+"savingrest.pkl",'wb') as f:
            pickle.dump(yhat, f)
        with open("xxxx"+test_name+"/"+test_name+str(days)+'/'+str(i)+"other.pkl",'wb') as f:
            pickle.dump(y_testw, f) 
        errrmse.append(rmse)
        errmae.append(mae)
        errmse.append(mse)
        errrpear.append(cor)
        errr2.append(r2)

        np.savetxt("xxxx"+test_name+"/"+test_name+str(days)+"/"+test_name+"round"+str(i)+".csv", [rmse,mae,mse,cor,r2], delimiter=",")

      mediaremse = statistics.mean(errrmse)
      mediamae = statistics.mean(errmae)
      mediamse = statistics.mean(errmse)
      mediamr2 = statistics.mean(errr2)
      meadipear = statistics.mean(errrpear)
      errors.append([mediaremse,mediamae,mediamse,mediamr2,meadipear])
      arra = np.array(errors)
      np.savetxt("xxxx"+test_name+"/"+test_name+str(days)+"/"+test_name+"total.csv", arra, delimiter=",")
    

In [ ]:
to_train1 = pd.read_csv('xxxx.csv')
result = pd.read_csv('xxx.csv')
try_df = pd.read_csv('/content/drive/MyDrive/Luis/experiments_prediction_fake_news/data/try_df.csv')
prueba = to_train1.drop(columns=[ 'mean_neutral', 'max_neutral', 'min_neutral', 'count_neutral', 'mean_negative', 'max_negative', 'min_negative', 'mean_positive', 'max_positive', 'min_positive', 'max_IsRootEvent', 'min_IsRootEvent',])
prueba['just_dates'] = pd.to_datetime(prueba['just_dates'])
prueba['dates'] = pd.to_datetime(prueba['dates'])

In [ ]:
for percentage_train in [0.6,0.9]:
  for j in [1,2,4,7]:
    days_to_predict = j
    epochs = 3000
    batch_size = 512
    texts_to_input = 0
    percentage_training = percentage_train
    consistency = 1
    test_name = "xxxxx" +str(days_to_predict)+"_"+str(epochs)+"_"+str(batch_size)+"_"+str(percentage_training)
    try_df = count_fakes_in_next_days(prueba,result,days_to_predict)
    try_df = mc.MoveToLast(try_df,'number_fake_news')
    try_df = try_df[try_df['dates']>= "2019-10-01"]
    try_df = try_df[try_df['dates']<= "2022-07-14"]
    dates_df = try_df['dates']
    try_df = try_df.drop(columns=["just_dates","dates"])

    train = try_df
    scalers={}
    for i in try_df.columns:
     
          scaler = MinMaxScaler()
          s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
          s_s=np.reshape(s_s,len(s_s))
          scalers['scaler_'+ i] = scaler
          train[i]=s_s

    train['dates'] = dates_df
    if consistency:
      
      testing_consistency = train[train['dates']>= "2022-03-01"]
      testing_consistency = testing_consistency[testing_consistency['dates']<= "2022-03-30"]
      dates_df = testing_consistency['dates']
      testing_consistency = testing_consistency.drop(columns=["dates"])
      

      testing_consistency['dates'] = dates_df
      train = train[~train.dates.isin(testing_consistency.dates)]
      testing_consistency = mc.MoveToLast(testing_consistency,'number_fake_news')

    results_experiments = []
    errors = []
    try_df = mc.MoveToLast(try_df,'number_fake_news')
    
    for days in [1,2,3,4,5,6,7,8,9,10]:
      err = []
      errrmse = []
      errmse = []
      errmae = []
      errr2= []
      errrpear = []
      partial_results = []
      ys = None
      for i in range(5):
  
        path = "xxx"+test_name+"/"+test_name+str(days)+"/"
        if not os.path.exists("xxx"+test_name+"/"):
          os.mkdir("xxx"+test_name+"/")

        tf.keras.utils.set_random_seed(i) 
        dataset,target_values,list_days = create_dataset_target_days(train,days)
        dataset = np.array(dataset)
        target_values = np.array(target_values)
      
        X_test,X_train,y_test,y_train,scaler = prepare_data(dataset,target_values,list_days,percentage_training,i)

        if consistency:
          dataset_check,target_values_check,list_days = create_dataset_target_days(testing_consistency,days)
          dataset_check = np.array(dataset_check)
          target_values_check = np.array(target_values_check)
          consistency_x,consistency_y = prepare_data_consistency(dataset_check,target_values_check,scaler)
        ys = y_test

        if not os.path.exists(path):
          os.mkdir(path)
      

        model = autoencoder(X_train)
        early_stopping_monitor = EarlyStopping(
      monitor='val_loss',
      min_delta=0,
      patience=1000,
      verbose=0,
      mode='auto',
      baseline=None,
      restore_best_weights=True
  )

        if consistency == 1:
          history = model.fit(X_train, y_train, epochs=epochs, validation_data=(consistency_x,consistency_y ), batch_size=batch_size, verbose=0,callbacks=[early_stopping_monitor], shuffle=True)
        else:
          history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),callbacks=[early_stopping_monitor], verbose=0, shuffle=True)

      
        if consistency:
          yhat = model.predict(consistency_x)
          yhat = yhat.flatten()
          consistency_y = consistency_y.flatten()
          consistency_y = scalers['scaler_number_fake_news'].inverse_transform(consistency_y.reshape(-1, 1)).flatten()
          yhat = scalers['scaler_number_fake_news'].inverse_transform(yhat.reshape(-1, 1)).flatten()
          yhat[yhat < 0] = 0
          rmse = sqrt(mean_squared_error(consistency_y,yhat))
          mse = mean_squared_error(consistency_y,yhat)
          mae = np.mean(np.abs(consistency_y - yhat))
          r2 = r2_score(consistency_y, yhat)  
          cor = np.corrcoef(consistency_y, yhat)[0][1] 
          

          with open("xxxx"+test_name+"/"+test_name+str(days)+'/'+str(i)+"xxx.pkl",'wb') as f:
            pickle.dump(yhat, f)
          with open("xxx/"+test_name+"/"+test_name+str(days)+'/'+str(i)+"xxx.pkl",'wb') as f:
            pickle.dump(consistency_y, f) 

        else:

          yhat = model.predict(X_test)
          yhat = yhat.flatten()
          y_test = y_test.flatten()
          y_test = scalers['scaler_number_fake_news'].inverse_transform(y_test.reshape(-1, 1)).flatten()
          yhat = scalers['scaler_number_fake_news'].inverse_transform(yhat.reshape(-1, 1)).flatten()
          yhat[yhat < 0] = 0
      
          rmse = sqrt(mean_squared_error(y_test,yhat))
          mse = mean_squared_error(y_test,yhat)
          mae = np.mean(np.abs(y_test - yhat))
          r2 = r2_score(y_test, yhat)


          with open("xxxx"+test_name+"/"+test_name+str(days)+'/'+str(i)+"xxxx.pkl",'wb') as f:
            pickle.dump(yhat, f)
          with open("xxxx"+test_name+"/"+test_name+str(days)+'/'+str(i)+"xxxx.pkl",'wb') as f:
            pickle.dump(y_test, f) 

        

        partial_results.append(yhat[0])

        errrmse.append(rmse)
        errmae.append(mae)
        errmse.append(mse)
        errrpear.append(cor)
        errr2.append(r2)

        
        plt.figure(figsize=(80,10))
        if consistency:
          plt.plot(yhat,label='Prediction',alpha=0.8, linestyle="dotted")
          plt.plot(consistency_y,label='Real')
        else:
          plt.plot(yhat,label='Prediction',alpha=0.8, linestyle="dotted")
          plt.plot(y_test,label='Real')

        plt.legend(loc="upper left")
        plt.savefig("xxxx"+test_name+"/"+test_name+str(days)+'/'+str(i)+"_"+str(days)+'.png')
      plt.show()


      plt.figure(figsize=(80,10))
      if consistency:
        plt.plot(consistency_y,label='Real',linewidth=2.0)
      else:
        plt.plot(y_test,label='Real',linewidth=2.0)

      avgs = np.squeeze(np.average(partial_results, axis=0))
      plt.plot(avgs,label='Avg predictions',alpha=0.6)
      mins = np.squeeze(np.min(partial_results, axis=0))
      maxs = np.squeeze(np.max(partial_results, axis=0))
      xs = []
      if consistency:
        for i in range(len(consistency_y)):
          xs.append(i)
      else:
        for i in range(len(y_test)):
          xs.append(i)

      plt.plot(mins,alpha=0.5)
      plt.plot(maxs,alpha=0.5)
      plt.fill_between(xs,mins,maxs,alpha=0.2,interpolate=True,color="grey")
      plt.legend(loc="upper left")
      plt.savefig("xxxx"+test_name+"/"+test_name+str(days)+'/'+"TOTAL"+str(days)+'.png')

      results_experiments.append(partial_results)
      mediaremse = statistics.mean(errrmse)
      mediamae = statistics.mean(errmae)
      mediamse = statistics.mean(errmse)
      mediamr2 = statistics.mean(errr2)
      meadipear = statistics.mean(errrpear)
  
      errors.append([mediaremse,mediamae,mediamse,mediamr2,meadipear])
      arra = np.array(errors)
      np.savetxt("xxxx"+test_name+"/"+test_name+str(days)+"/"+test_name+".csv", arra, delimiter=",")

    arra = np.array(errors)
    np.savetxt("xxx"+test_name+"/"+test_name+str(days)+"/"+test_name+".csv", arra, delimiter=",")
    np.savetxt("/xxxx/"+test_name+".csv", arra, delimiter=",")

# Predicion on a date

In [ ]:
def count_fakes_in_next_days_prediction(df,df_maldita,days):
  dates = df['dates'].tolist()
  len_dates = []
  predictions = []

  for date in dates:  
    grupo = []
    for dia in range(days):
      objetivo = dia
      d = date + timedelta(days=objetivo)
      str_d = str(d).split(" ")[0]
      original_date = str(date).split(" ")[0]
      testing_week = df_maldita[(df_maldita['just_date'] == str_d)]
      grupo.append(len(testing_week))
    
    predictions.append(grupo)
  return predictions


def count_fakes_in_next_days_prediction_co(df,df_maldita,days,used):
  dates = df['dates'].tolist()
  len_dates = []
  predictions = []
  for date in dates:
  
    grupo = []
    for dia in range(days):
      objetivo = dia+used+1
      d = date + timedelta(days=objetivo)
      str_d = str(d).split(" ")[0]
      original_date = str(date).split(" ")[0]
      testing_week = df_maldita[(df_maldita['just_date'] == str_d)]
      grupo.append(len(testing_week)) 
    predictions.append(grupo)
  return predictions

In [ ]:

for prediction in [2,4,7]:
  errors = []
  prueba = to_train1[to_train1['dates']>= "2019-10-01"]
  prueba = prueba[prueba['dates']<= "2022-07-14"]
  prueba = prueba.drop(columns=[ 'mean_neutral', 'max_neutral', 'min_neutral', 'count_neutral', 'mean_negative', 'max_negative', 'min_negative', 'mean_positive', 'max_positive', 'min_positive', 'max_IsRootEvent', 'min_IsRootEvent'])
  dates_df = prueba['dates']
  days_to_predict = prediction
  epochs = 3000
  batch_size = 1000
  texts_to_input = 0
  percentage_training = 1
  consistency = 1
  seed = 1
  for days_window in [4,9,14,19]:
    used = days_window
    test_name = "xxxx" +str(days_to_predict)+"_"+str(epochs)+"_"+str(batch_size)+"_"+str(percentage_training)
    path = "xxxx"+test_name+"/"+test_name+str(days_window)+"/"
    if not os.path.exists("xxxx"+test_name+"/"):
          os.mkdir("xxx"+test_name+"/")
          os.mkdir("xxx"+test_name+"/"+test_name+str(days_window)+"/")
    if not os.path.exists("xxxx"+test_name+"/"+test_name+str(days_window)+"/"):
        
          os.mkdir("/xxx"+test_name+"/"+test_name+str(days_window)+"/")
     
    lista_dias = count_fakes_in_next_days_prediction_co(prueba,result,days_to_predict,used)
    prueba['dates'] = dates_df
    prueba['just_dates'] = dates_df

    prueba = count_fakes_in_next_days(prueba,result,1)
    prueba = prueba.drop(columns=["just_dates","dates"])

    train = prueba
    scalers={}
    for i in prueba.columns:
      if not 'scaler_'+ i == "scaler_number_fake_news":
          scaler = MinMaxScaler()
          s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
          s_s=np.reshape(s_s,len(s_s))
          scalers['scaler_'+ i] = scaler
          train[i]=s_s
    train['dates'] = dates_df
    if consistency: 
      datetimenew = datetime.strptime("2022-03-30", '%Y-%m-%d')
      end_date = datetimenew 
      datetimenew = datetime.strptime("2022-03-01", '%Y-%m-%d')
      start = datetimenew

      testing_consistency = train[train['dates']>= start]
      end_datestr = str(end_date).split(" ")
      testing_consistency = testing_consistency[testing_consistency['dates']<= end_datestr[0]]

      train = train[~train.dates.isin(testing_consistency.dates)]
      lista_dias_consistencia = count_fakes_in_next_days_prediction_co(testing_consistency,result,days_to_predict,used)


    dataset,list_days = create_dataset_target_days(train,used)
    dataset = np.array(dataset)
    dif = abs(len(dataset) - len(lista_dias))
    lista_dias = lista_dias[dif:]
    target_values = lista_dias
    X_test,X_train,y_test,y_train,scaler = prepare_data(dataset,target_values,list_days,percentage_training,seed)
    X_train = np.array(X_train)


    if consistency:
          dataset_check,list_days_con = create_dataset_target_days(testing_consistency,used)
          dataset_check = np.array(dataset_check)
          lista_dias_consistencia = np.array(lista_dias_consistencia)
          dif = abs(len(dataset_check) - len(lista_dias_consistencia))
          lista_dias_consistencia = lista_dias_consistencia[dif:]
          X_test_con,X_train_con,y_test_con,y_train_con = prepare_data_consistency(dataset_check,lista_dias_consistencia,list_days,1)


    rmse_to = []
    mse_to = []
    mae_to = []
    r2_to = []
    for round in range(5):
      tf.keras.utils.set_random_seed(round)
      model = create_autoencoders(X_train,y_train)
      early_stopping_monitor = EarlyStopping(
          monitor='val_loss',
          min_delta=0,
          patience=1000,
          verbose=0,
          mode='auto',
          baseline=None,
          restore_best_weights=True
      )

      if consistency == 1:
          history = model.fit(X_train, y_train, epochs=epochs, validation_data=( X_train_con,y_train_con ), batch_size=1000, verbose=0,callbacks=[early_stopping_monitor], shuffle=True)
      else:
          history = model.fit(X_train, y_train, epochs=epochs, batch_size=1000, validation_data=(X_test, y_test),callbacks=[early_stopping_monitor], verbose=0, shuffle=True)


      yhat = model.predict(X_train_con)
      predic = yhat.flatten()
      ground = y_train_con.flatten()
      rmse = sqrt(mean_squared_error(ground,predic))
      mse = mean_squared_error(ground,predic)
      mae = np.mean(np.abs(ground - predic))
      r2 = r2_score(ground, predic)  
      np.savetxt("xxxx"+test_name+"/"+test_name+str(days_window)+"/"+str(round)+"xx.csv", [rmse,mse,mae,r2], delimiter=",")

      rmse_to.append(rmse)
      mse_to.append(mse)
      mae_to.append(mae)
      r2_to.append(r2)
      save = []
      save.append([rmse,mse,mae,r2])
      arra = np.array(save)
      np.savetxt("xxxx"+test_name+"/"+test_name+str(days_window)+"/"+str(round)+"xxx.csv", arra, delimiter=",")

    mediaremse = statistics.mean(rmse_to)
    mediamae = statistics.mean(mae_to)
    mediamse = statistics.mean(mse_to)
    mediamr2 = statistics.mean(r2_to)
    
    errors.append([mediaremse,mediamae,mediamse,mediamr2])
    arra = np.array(errors)
    np.savetxt("xxxx"+test_name+"/"+test_name+str(days_window)+"/"+"xxx.csv", arra, delimiter=",")


In [ ]:
for prediction in [2,4,7]:
  errors = []
  prueba = to_train1[to_train1['dates']>= "2019-10-01"]
  prueba = prueba[prueba['dates']<= "2022-07-14"]
  prueba = prueba.drop(columns=[ 'mean_neutral', 'max_neutral', 'min_neutral', 'count_neutral', 'mean_negative', 'max_negative', 'min_negative', 'mean_positive', 'max_positive', 'min_positive', 'max_IsRootEvent', 'min_IsRootEvent'])
  dates_df = prueba['dates']
  days_to_predict = prediction
  epochs = 3000
  batch_size = 512
  texts_to_input = 0
  percentage_training = 1
  consistency = 1

  for days_window in [4,9,14,19]:
    try_df = count_fakes_in_next_days(to_train1,result,days_to_predict)
    try_df = mc.MoveToLast(try_df,'number_fake_news')
    try_df = try_df[try_df['dates']>= "2019-10-01"]
    try_df = try_df[try_df['dates']<= "2022-07-14"]
    dates_df = try_df['dates']
    test_name = "marzo_contex" +str(days_to_predict)+"_"+str(epochs)+"_"+str(batch_size)+"_"+str(percentage_training)

    try_df = try_df.drop(columns=["just_dates","dates"])

    train = try_df
    scalers={}
    for i in try_df.columns:
        
          scaler = MinMaxScaler(feature_range=(0,1))
          s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
          s_s=np.reshape(s_s,len(s_s))
          scalers['scaler_'+ i] = scaler
          train[i]=s_s

    train['dates'] = dates_df
    if consistency:
      given_date = '2022-03-01'
      date_format = '%Y-%m-%d'

      dtObj = datetime.strptime(given_date, date_format)
      d = dtObj- timedelta(days=days_window)
      testing_consistency = train[train['dates']>= d]

      testing_consistency = testing_consistency[testing_consistency['dates']<= "2022-03-30"]
      dates_df = testing_consistency['dates']
      testing_consistency = testing_consistency.drop(columns=["dates"])
      testing_consistency['dates'] = dates_df
      train = train[~train.dates.isin(testing_consistency.dates)]
      testing_consistency = mc.MoveToLast(testing_consistency,'number_fake_news')      



    err = []
    errrmse = []
    errmse = []
    errmae = []
    errr2= []
    days = days_window
    errrpear = []
    errors = []
    used = days_window
    for i in range(5):
      results_experiments = []
      try_df = mc.MoveToLast(train,'number_fake_news')
      listdays = try_df['dates'].tolist()
      lista_dias = count_fakes_in_next_days_prediction_co(try_df,result,days_to_predict,used)

      dias_tr = train['dates'].to_list()
      df_filtered = training_gdelt[training_gdelt['dates'].isin(dias_tr)]
      texts_in_dataset = group_titles_by_day(listdays,df_filtered)
      texts_by_day = []
      path = "xxx"+test_name+"/"+test_name+str(days)+"/"
      if not os.path.exists("xxxx"+test_name+"/"):
        os.mkdir("xxx"+test_name+"/")
      if not os.path.exists("xxx"+test_name+"/"+test_name+str(days)+"/"):
        os.mkdir("xxx"+test_name+"/"+test_name+str(days)+"/")
      print("saving in: "+ path)
      for textday in texts_in_dataset:
        sub_arra = []
        i = 0
        while i < 10:
          if len(textday) < 10:
            for x in range(len(textday)):
              sub_arra += textday[x]
              i += 1
            
            howmany = abs(len(sub_arra)-10)
            for y in range(howmany):
              sub_arra.append("")
          else:
            sub_arra.append(textday[i])
            i += 1
        
        texts_by_day.append(sub_arra)
      


      dias_tr = testing_consistency['dates'].to_list()
      df_filtered2 = training_gdelt[training_gdelt['dates'].isin(dias_tr)]
      listdays = testing_consistency['dates'].tolist()
      texts_in_dataset2 = group_titles_by_day(listdays,df_filtered2)
      texts_by_day2 = []
      
       
      path = "xxx"+test_name+"/"+test_name+str(days)+"/"
      if not os.path.exists("xxx"+test_name+"/"):
        os.mkdir("xxx"+test_name+"/")
      if not os.path.exists("xxx"+test_name+"/"+test_name+str(days)+"/"):
        os.mkdir("xxx"+test_name+"/"+test_name+str(days)+"/")
      print("saving in: "+ path)
      for textday in texts_in_dataset2:
        sub_arra = []
        i = 0
        while i < 10:
          if len(textday) < 10:
            for x in range(len(textday)):
              sub_arra += textday[x]
              i += 1
            
            howmany = abs(len(sub_arra)-10)
            for y in range(howmany):
              sub_arra.append("")
          else:
            sub_arra.append(textday[i])
            i += 1
        
        texts_by_day2.append(sub_arra)
      lista_dias2 = count_fakes_in_next_days_prediction_co(testing_consistency,result,days_to_predict,used)
      target_days = i
      dataset,target_values,list_days,aa = create_dataset_text(try_df,target_days,1,texts_by_day)
      dataset2,target_values2,list_days2,aa2 = create_dataset_text(testing_consistency,target_days,1,texts_by_day2)
      difs = abs(len(dataset)-len(aa))
      aa = aa[:len(aa)-difs]
      aa2 = aa2[:len(aa2)-target_days]
      dif = abs(len(dataset) - len(lista_dias))
      lista_dias = lista_dias[:len(lista_dias)-dif]
      dif = abs(len(dataset2) - len(lista_dias2))
      lista_dias2 = lista_dias2[:len(lista_dias2)-dif]

      dataset = np.array(dataset)
      target_values = np.array(target_values)
      X_test,X_train,y_test,y_train,scaler,test_texts,train_texts = prepare_data_with_texts(dataset,lista_dias,list_days,aa,percentage_training)
      dataset2 = np.array(dataset2)
      target_values2 = np.array(target_values2)
      X_test2,X_train2,y_test2,y_train2,scaler2,test_texts2,train_texts2 = prepare_data_with_texts_no(dataset2,lista_dias2,list_days2,aa2,percentage_training)

      train_text_new = np.array(train_texts)
      test_text_new = np.array(train_texts2)
      train_1 = []
      train_2 = []
      train_3 = []
      train_4 = []
      train_5 = []
      train_6 = []
      train_7 = []
      train_8 = []
      train_9 = []
      train_10 = []

      test_1 = []
      test_2 = []
      test_3 = []
      test_4 = []
      test_5 = []
      test_6 = []
      test_7 = []
      test_8 = []
      test_9 = []
      test_10 = []

      for group in train_text_new:
        train_1.append(group[0])
        train_2.append(group[1])
        train_3.append(group[2])
        train_4.append(group[3])
        train_5.append(group[4])
        train_6.append(group[5])
        train_7.append(group[6])
        train_8.append(group[7])
        train_9.append(group[8])
        train_10.append(group[9])

      for group in test_text_new:
        test_1.append(group[0])
        test_2.append(group[1])
        test_3.append(group[2])
        test_4.append(group[3])
        test_5.append(group[4])
        test_6.append(group[5])
        test_7.append(group[6])
        test_8.append(group[7])
        test_9.append(group[8])
        test_10.append(group[9])

      train_1 = np.array(train_1)
      train_2 = np.array(train_2)
      train_3 = np.array(train_3)
      train_4 = np.array(train_4)
      train_5 = np.array(train_5)
      train_6 = np.array(train_6)
      train_7 = np.array(train_7)
      train_8 = np.array(train_8)
      train_9 = np.array(train_9)
      train_10 = np.array(train_10)


      test_1 = np.array(test_1)
      test_2 = np.array(test_2)
      test_3 = np.array(test_3)
      test_4 = np.array(test_4)
      test_5 = np.array(test_5)
      test_6 = np.array(test_6)
      test_7 = np.array(test_7)
      test_8 = np.array(test_8)
      test_9 = np.array(test_9)
      test_10 = np.array(test_10)

      early_stopping_monitor = EarlyStopping(
          monitor='val_loss',
          min_delta=0,
          patience=1000,
          verbose=0,
          mode='auto',
          baseline=None,
          restore_best_weights=True
      )
      model1 = create_model_text(X_train,vectorize_layer_for_model,y_train)
      history = model1.fit([X_train,train_1,train_2,train_3,train_4,train_5,train_6,train_7,train_8,train_9,train_10], y_train, epochs=3000, batch_size=batch_size,callbacks=[early_stopping_monitor],validation_data=([X_train2,test_1,test_2,test_3,test_4,test_5,test_6,test_7,test_8,test_9,test_10], y_train2), verbose=0, shuffle=True)
      yhat = model1.predict([X_train2,test_1,test_2,test_3,test_4,test_5,test_6,test_7,test_8,test_9,test_10])
      yhat = yhat.flatten()
      y_testw = y_train2.flatten()

      yhat[yhat < 0] = 0
      rmse = sqrt(mean_squared_error(y_testw,yhat))
      mse = mean_squared_error(y_testw,yhat)
      mae = np.mean(np.abs(y_testw - yhat))
      r2 = r2_score(y_testw, yhat)
      cor = np.corrcoef(y_testw, yhat)[0][1] 

      errrmse.append(rmse)
      errmae.append(mae)
      errmse.append(mse)
      errrpear.append(cor)
      errr2.append(r2)

      np.savetxt("xxxx"+test_name+"/"+test_name+str(days)+"/"+test_name+"round"+str(i)+".csv", [rmse,mae,mse,cor,r2], delimiter=",")

    mediaremse = statistics.mean(errrmse)
    mediamae = statistics.mean(errmae)
    mediamse = statistics.mean(errmse)
    mediamr2 = statistics.mean(errr2)
    meadipear = statistics.mean(errrpear)
    errors.append([mediaremse,mediamae,mediamse,mediamr2,meadipear])
    arra = np.array(errors)
    np.savetxt("xxxx"+test_name+"/"+test_name+str(days)+"/"+test_name+"total.csv", arra, delimiter=",")



       

